In [1]:
import pandas as pd
from geopy.geocoders import ArcGIS
arc=ArcGIS()

In [2]:
df = pd.read_csv("datasets/COVID-19-geographic-disbtribution-worldwide-2020-03-31.csv")
df.head

<bound method NDFrame.head of          dateRep  day  month  year  cases  deaths countriesAndTerritories  \
0     31/03/2020   31      3  2020     27       0             Afghanistan   
1     30/03/2020   30      3  2020      8       1             Afghanistan   
2     29/03/2020   29      3  2020     15       1             Afghanistan   
3     28/03/2020   28      3  2020     16       1             Afghanistan   
4     27/03/2020   27      3  2020      0       0             Afghanistan   
...          ...  ...    ...   ...    ...     ...                     ...   
7899  25/03/2020   25      3  2020      0       0                Zimbabwe   
7900  24/03/2020   24      3  2020      0       1                Zimbabwe   
7901  23/03/2020   23      3  2020      0       0                Zimbabwe   
7902  22/03/2020   22      3  2020      1       0                Zimbabwe   
7903  21/03/2020   21      3  2020      1       0                Zimbabwe   

     geoId countryterritoryCode  popData2018 

In [6]:
loc = arc.geocode("ZW")

In [7]:
loc.latitude

-19.016205470999978

In [11]:
lat = []
long = []
code = {}
for country in df.geoId.unique():
    try:
        loc = arc.geocode(country)
        code[country] = {"lat":loc.latitude, "long":loc.longitude}
    except:
        code[country] = {"lat":"nan", "long":"nan"}
        
    

In [16]:
df_country = pd.DataFrame({"country":list(code.keys()), "lat":[code[i]["lat"] for i in code.keys()], "long":[code[i]["long"] for i in code.keys()]})

In [24]:
df_country.to_csv("encodedcountrynames.csv")

In [25]:
df_dict = df_country.to_dict()

In [29]:
import json 
with open("countryloc.json","w") as file:
    json.dump(df_dict, file)

In [30]:
with open("countryloc.json","r") as file:
    c = json.load(file)

In [34]:
df_merge = df.merge(df_country, left_on = "geoId", right_on = "country", how = "inner")

In [35]:
df_merge

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018,country,lat,long
0,31/03/2020,31,3,2020,27,0,Afghanistan,AF,AFG,37172386.0,AF,33.8311,66.0247
1,30/03/2020,30,3,2020,8,1,Afghanistan,AF,AFG,37172386.0,AF,33.8311,66.0247
2,29/03/2020,29,3,2020,15,1,Afghanistan,AF,AFG,37172386.0,AF,33.8311,66.0247
3,28/03/2020,28,3,2020,16,1,Afghanistan,AF,AFG,37172386.0,AF,33.8311,66.0247
4,27/03/2020,27,3,2020,0,0,Afghanistan,AF,AFG,37172386.0,AF,33.8311,66.0247
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7899,25/03/2020,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0,ZW,-19.0162,29.8843
7900,24/03/2020,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14439018.0,ZW,-19.0162,29.8843
7901,23/03/2020,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0,ZW,-19.0162,29.8843
7902,22/03/2020,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0,ZW,-19.0162,29.8843


In [50]:
df_merge_ave = df_merge.groupby("geoId").agg("sum")

In [52]:
df_merge_ave.reset_index(inplace = True)


In [59]:
df_merge2 = df_merge_ave.merge(df_merge[["lat","long","geoId"]], on = "geoId", how = "left")

In [61]:
df_merge2 = df_merge2.groupby("geoId").first()

In [63]:
df_merge2.reset_index(inplace = True)

In [67]:
df_merge3 = df_merge2[df_merge2.long!="nan"]
df_merge3

,geoId,day,month,year,cases,deaths,popData2018,lat,long
0,AD,393,54,36360,370,8,1.386108e+06,42.5453,1.57629
1,AE,1408,176,173719,611,5,8.282625e+08,23.9149,54.3269
2,AF,1375,164,165639,141,4,3.048136e+09,33.8311,66.0247
3,AG,301,36,24240,7,0,1.155432e+06,21.8812,-102.296
4,AI,145,15,10100,2,0,0.000000e+00,18.2246,-63.0596
...,...,...,...,...,...,...,...,...,...
193,VN,1440,182,177759,229,0,8.407555e+09,16.1667,107.833
194,XK,376,48,32320,106,1,2.952480e+07,42.5815,20.8916
195,ZA,463,72,48480,1326,3,1.386711e+09,-28.9972,25.085
196,ZM,325,39,26260,35,0,2.255737e+08,-14.4688,28.768


In [105]:
import folium

In [106]:
maps = folium.Map(location = [36.1699, -115.1398], zoom_start=5, tiles= "stamen terrain")

In [107]:
FGd = folium.FeatureGroup("deaths")
for death, long, lat, g in zip(df_merge3.deaths.tolist(),df_merge3.long.tolist(),df_merge3.lat.tolist(), df_merge3.geoId.tolist()):
    FGd.add_child(folium.Marker([lat, long], tooltip = f"{g} Number of deaths: {death}"))




In [108]:
FGc = folium.FeatureGroup("cases")
for c, long, lat, g in zip(df_merge3.cases.tolist(),df_merge3.long.tolist(),df_merge3.lat.tolist(), df_merge3.geoId.tolist()):
    FGc.add_child(folium.Marker([lat, long],icon = folium.Icon(color="green"), tooltip = f"{g} Number of cases: {c}"))




In [109]:
df_merge3["mortalityrate"]=round(df_merge3.deaths/df_merge3.cases, 2)

C:\Users\ERCL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [110]:
FGm = folium.FeatureGroup("mortalityrate")
for m, long, lat, g in zip(df_merge3.mortalityrate.tolist(),df_merge3.long.tolist(),df_merge3.lat.tolist(), df_merge3.geoId.tolist()):
    FGm.add_child(folium.Marker([lat, long],icon = folium.Icon(color="red"),  tooltip = f"{g} mortality rate: {m}"))

maps.add_child(FGd)
maps.add_child(FGc)
maps.add_child(FGm)
maps.add_child(folium.LayerControl())


In [111]:
maps.save("coronamap.html")